In [76]:
from pyquery import PyQuery
import re
import time
import sqlite3


def get_brand(code):
    url = "https://kabutan.jp/stock/?code={}".format(code)
    
    q = PyQuery(url)
    if len(q.find('#kobetsu_right')('span')) == 0:
        return None
    
    try:
        code = q('#stockinfo_b0')('h2')('span').text()
        name = q('#kobetsu_right')('h3').text()
        short_name = q('#stockinfo_b0')('h2').text()[4:]
        market = q('#stockinfo_b0')('span.market').text()
        sector = q('#stockinfo_i2')('a').text()
        unit_str = q('#stockinfo_i2')('dd').text()
        unit = int(re.sub("\\D", "", unit_str))
    except ValueError:
        return None
    
    return code, name, short_name, market, sector, unit

def brands_generator(code_range):
    for code in code_range:
        brand = get_brand(code)
        if brand:
            yield brand
        time.sleep(1)

def insert_brands_to_db(db_file_name, code_range):
    conn = sqlite3.connect(db_file_name)
    with conn:
        sql = 'INSERT INTO brands(code, name, short_name, market, sector, unit)' \
                 'VALUES(?, ?, ?, ?, ?, ?)'
        conn.executemany(sql, brands_generator(code_range))

# example
get_brand(1301)

('1301', '極洋', '極洋', '東証１', '水産・農林業', 100)